# Collect and Reduce Classifier Embeddings

In this tutorial, we will use an existing classifier model to generate per-bounding-box embeddings for a COCO-style object detection dataset. We will then reduce these embeddings to 3D using PaCMAP.

To run this notebook, you must also have run:
* [1-fine-tune-on-crops.ipynb](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/bb-embeddings/1-fine-tune-on-crops.ipynb)

## Project Setup

In [ ]:
PROJECT_NAME = "3LC Tutorials"
TRANSIENT_DATA_PATH = "../../../transient_data"
BATCH_SIZE = 32
DATASET_NAME = "COCO128"
DEVICE = None
INSTALL_DEPENDENCIES = False

In [ ]:
%%capture
if INSTALL_DEPENDENCIES:
    %pip --quiet install pacmap

## Imports

In [ ]:
from io import BytesIO

import numpy as np
import pacmap
import timm
import tlc
import torch
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm

## Set device

In [ ]:
if DEVICE is None:
    if torch.cuda.is_available():
        device = "cuda:0"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
else:
    device = DEVICE

device = torch.device(device)
print(f"Using device: {device}")

## Get Input Table

In [ ]:
# Open the Table created in the previous notebook
input_table = tlc.Table.from_names(
    project_name=PROJECT_NAME, 
    dataset_name=DATASET_NAME, 
    table_name="initial"
)

# Get the schema of the bounding box list, required to crop
bb_schema = input_table.rows_schema.values["bbs"].values["bb_list"]

## Get Model

In [ ]:
# Load the model trained in the previous notebook
model = timm.create_model(
    "efficientnet_b0", 
    num_classes=81, 
    checkpoint_path=TRANSIENT_DATA_PATH + "/bb_classifier.pth"
).to(device)

model.eval()

# The hidden layer whose activations we will use for embeddings
hidden_layer = model.global_pool.flatten

## Set Up Data Processing

In [ ]:
# The transformation to apply to the image before feeding it to the model
image_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# An iterator over every transformed BB crop in a single sample (image)
def single_sample_bb_crop_iterator(sample):
    image_filename = sample["image"]
    image_bytes = tlc.Url(image_filename).read()
    image = Image.open(BytesIO(image_bytes))
    w, h = image.size

    for bb in sample["bbs"]["bb_list"]:
        bb_crop = tlc.BBCropInterface.crop(image, bb, bb_schema, h, w)
        yield image_transform(bb_crop)

# An iterator over every transformed BB crop in the dataset
def bb_crop_iterator():
    for sample in input_table:
        for bb_crop in single_sample_bb_crop_iterator(sample):
            yield bb_crop

# A batched iterator over every transformed BB crop in the dataset
def batched_bb_crop_iterator():
    batch = []
    for bb_crop in bb_crop_iterator():
        batch.append(bb_crop)
        if len(batch) == BATCH_SIZE:
            yield torch.stack(batch).to(device)
            batch = []
    if batch:
        yield torch.stack(batch).to(device)

## Collect Embeddings

In [ ]:
# Add a model hook which saves activations to output_list
output_list = []
def hook_fn(module, input, output):
    output_list.append(output.cpu())

hook_handle = hidden_layer.register_forward_hook(hook_fn)

In [ ]:
# Use our iterator to run the model on every BB crop in the dataset
for batch in tqdm(batched_bb_crop_iterator(), desc="Running model inference"):
    with torch.no_grad():
        model(batch)

hook_handle.remove()

## Dimensionality Reduction

In [ ]:
# Stack all the embeddings into a single numpy array
embeddings = np.vstack(output_list)

# Reduce the 1280-dimensional activations to 3D using pacmap
reducer = pacmap.PaCMAP(n_components=3)
embeddings_3d = reducer.fit_transform(embeddings)

In [ ]:
# Save embeddings for use in the next notebook(s)
np.save(TRANSIENT_DATA_PATH + "/bb_classifier_embeddings.npy", embeddings_3d)